# Prediction of the area under the curve (AUC) of the drug release profile with Linear Regression (LinR)
## Initialization of environment

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import optuna
import shap
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
import warnings
optuna.logging.set_verbosity(optuna.logging.ERROR)
warnings.filterwarnings("ignore")

## Data loading and preparation¶
Definition of variables, data loading, normalization and interpolation of the drug release profile, calculation of drug release profile AUC

In [6]:
num_interp_pts = 11
n_outer_folds = 10
n_inner_folds = 2
n_trials = 50
# ----------------------------------------------------------------------------------------
# Load data
# ----------------------------------------------------------------------------------------
file_path_form = 'mp_dataset_processed_no_dupes.xlsx'
file_path_time = 'mp_dataset_processed_time_release_only.xlsx'
formulation_df = pd.read_excel(file_path_form, engine='openpyxl')
release_df = pd.read_excel(file_path_time, engine='openpyxl')

# ----------------------------------------------------------------------------------------
# Encode categorical formulation variable
# ----------------------------------------------------------------------------------------
unique_values_emulsion = formulation_df['Formulation Method'].unique()
mapping = {v: i for i, v in enumerate(unique_values_emulsion)}
formulation_df['Formulation Method Encoded'] = formulation_df['Formulation Method'].map(mapping)
formulation_df.drop(columns=['Formulation Method', 'Drug SMILES'], inplace=True)


# ----------------------------------------------------------------------------------------
# Normalization and interpolation
# ----------------------------------------------------------------------------------------
group = release_df.groupby('Formulation Index')['Time']
min_time = group.transform('min')
max_time = group.transform('max')
release_df['Normalized Time'] = (release_df['Time'] - min_time) / (max_time - min_time)
normalized_times = np.linspace(0, 1, num_interp_pts)
interpolated_dfs = []
for formulation, g in release_df.groupby('Formulation Index'):
    g = g.sort_values('Time')
    time_min, time_max = g['Time'].min(), g['Time'].max()
    g['Normalized Time'] = (g['Time'] - time_min) / (time_max - time_min)
    interp_release = np.interp(normalized_times, g['Normalized Time'], g['Release'])
    interpolated_dfs.append(pd.DataFrame({
        'Formulation Index': formulation,
        'Normalized Time': normalized_times,
        'Interpolated Release': interp_release
    }))
interp_df = pd.concat(interpolated_dfs, ignore_index=True)

X = formulation_df.drop(columns=['Formulation Index']).to_numpy()
#X = formulation_df.to_numpy()  # [321, 11]
groups = interp_df.groupby('Formulation Index')['Interpolated Release']
y = np.stack([g.to_numpy().reshape(-1, 1) for _, g in groups]) # [321, 11, 1]

# ----------------------------------------------------------------------------------------
# AUC
# ----------------------------------------------------------------------------------------
auc = (
    interp_df.groupby("Formulation Index")
      .apply(lambda g: np.trapz(g["Interpolated Release"], g["Normalized Time"]))
      .reset_index(name="AUC")
)

y = auc['AUC'].values  # shape (n_samples,)



## Model Definition and Training

In [12]:
# ----------------------------------------------------------------------------------------
# Model wrapper for Linear Regression
# ----------------------------------------------------------------------------------------
class LinearModel:
    def __init__(self):
        self.model = LinearRegression()

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        return self.model.predict(X)

    def evaluate(self, X, y_true):
        y_pred = self.predict(X)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        return rmse

# ----------------------------------------------------------------------------------------
# Nested CV setup
# ----------------------------------------------------------------------------------------

outer_kf = KFold(n_splits=n_outer_folds, shuffle=True, random_state=42)

stored_best_models = []
stored_best_preds = []
stored_test_targets = []
stored_best_rmse = []
stored_best_mse = []
stored_r2 = []
stored_adj_r2 = []
stored_corr = []  
stored_pval = []  

# ----------------------------------------------------------------------------------------
# Outer CV loop
# ----------------------------------------------------------------------------------------
for outer_fold, (train_idx, test_idx) in enumerate(outer_kf.split(X, y)):
    print(f"\nOuter Fold {outer_fold + 1}")

    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    stored_test_targets.append(y_test)

    # ----------------------------------------------------------------------------------------
    # Scale features
    # ----------------------------------------------------------------------------------------
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # ----------------------------------------------------------------------------------------
    # Train model
    # ----------------------------------------------------------------------------------------
    model = LinearModel()
    model.fit(X_train, y_train)
    preds_best = model.predict(X_test)

    # ----------------------------------------------------------------------------------------
    # Compute metrics
    # ----------------------------------------------------------------------------------------
    mse_best = mean_squared_error(y_test, preds_best)
    rmse_best = np.sqrt(mse_best)
    r2_best = r2_score(y_test, preds_best)
    n = len(y_test)
    p = X_test.shape[1]
    adj_r2_best = 1 - (1 - r2_best) * (n - 1) / (n - p - 1)
    corr_best, pval_best = pearsonr(y_test, preds_best)
    # ----------------------------------------------------------------------------------------
    # Store results
    # ----------------------------------------------------------------------------------------
    stored_best_models.append(model)
    stored_best_preds.append(preds_best)
    stored_best_mse.append(mse_best)
    stored_best_rmse.append(rmse_best)
    stored_r2.append(r2_best)
    stored_adj_r2.append(adj_r2_best)
    stored_corr.append(corr_best)
    stored_pval.append(pval_best)

    print(
        f"Fold {outer_fold+1} "
        f"RMSE: {rmse_best:.4f}, "
        f"Corr: {corr_best:.4f}, "
        f"p-val: {pval_best:.2e}"
    )


Outer Fold 1
Fold 1 RMSE: 0.1416, Corr: 0.2861, p-val: 1.07e-01

Outer Fold 2
Fold 2 RMSE: 0.1088, Corr: 0.3703, p-val: 3.69e-02

Outer Fold 3
Fold 3 RMSE: 0.1518, Corr: 0.2892, p-val: 1.08e-01

Outer Fold 4
Fold 4 RMSE: 0.1385, Corr: -0.0712, p-val: 6.99e-01

Outer Fold 5
Fold 5 RMSE: 0.1385, Corr: 0.2139, p-val: 2.40e-01

Outer Fold 6
Fold 6 RMSE: 0.1535, Corr: 0.0915, p-val: 6.19e-01

Outer Fold 7
Fold 7 RMSE: 0.1409, Corr: 0.4010, p-val: 2.29e-02

Outer Fold 8
Fold 8 RMSE: 0.1111, Corr: 0.4503, p-val: 9.70e-03

Outer Fold 9
Fold 9 RMSE: 0.1817, Corr: 0.4325, p-val: 1.34e-02

Outer Fold 10
Fold 10 RMSE: 0.1509, Corr: 0.3442, p-val: 5.37e-02


## Performance metrics
RMSE, correlation, R-squared and adjusted R-squared

In [13]:
# --- Final metrics for Nested CV ---
rmse_mean, rmse_std = np.mean(stored_best_rmse), np.std(stored_best_rmse)
corr_mean, corr_std = np.mean(stored_corr), np.std(stored_corr)
pval_mean, pval_std = np.mean(stored_pval), np.std(stored_pval)

print(f"\n--- Final Nested CV Metrics ---")
print(f"RMSE: {rmse_mean:.4f} ± {rmse_std:.4f}")
print(f"Pearson correlation: {corr_mean:.4f} ± {corr_std:.4f}")
print(f"Pearson p-value: {pval_mean:.2e}")

# --- Overall metrics across all folds ---
y_true_all = np.concatenate(stored_test_targets)
y_pred_all = np.concatenate(stored_best_preds)

r_overall, pval_overall = pearsonr(y_true_all, y_pred_all)
r2_overall = r2_score(y_true_all, y_pred_all)

# Adjusted R²
n_samples = y_true_all.shape[0]  # or 321 if fixed
n_features = 11                  # or X.shape[1]
adj_r2_overall = 1 - (1 - r2_overall) * (n_samples - 1) / (n_samples - n_features - 1)

print(f"\n--- Overall Metrics ---")
print(f"Pearson r = {r_overall:.3f}, p-value = {pval_overall:.3g}")
print(f"R² = {r2_overall:.4f}, Adjusted R² = {adj_r2_overall:.4f}")



--- Final Nested CV Metrics ---
RMSE: 0.1417 ± 0.0199
Pearson correlation: 0.2808 ± 0.1557
Pearson p-value: 1.91e-01

--- Overall Metrics ---
Pearson r = 0.260, p-value = 2.28e-06
R² = 0.0505, Adjusted R² = 0.0167
